In [0]:
#importing libraries
import os
import sys
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

In [0]:
#BASE_DIR = r'D:\Work\DS_and_AI\Data_Sets'
#BASE_DIR = r'/content'
#GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
glove_file = r'/content/glove.6B.100d.txt'
#print(GLOVE_DIR)
#MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


In [10]:
#importing the dataset
!wget  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-04-14 12:13:24--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-04-14 12:13:24--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucde08cdce046448f16d461f2f0f.dl.dropboxusercontent.com/zip_download_get/AZPckEk_NikgxOh_qmebb3RVC-qf1LDflWzCJFT86CUOD8oz_NTSUg-0NJXBZC71i5XrItmf_PuI39vpCpkYO5B4DyIrdPYm-pXfPf-dOSoM6A [following]
--2020-04-14 12:13:25--  https://ucde08cdce046448f16d461f2f0f.dl.dropboxusercontent.com/zip_download_get/AZPckEk_NikgxOh_qmebb3RVC-qf1LDflWzCJFT86CUOD8oz_NTSUg-0NJXBZC71i5XrItmf_PuI39vpCpkY

In [11]:
#unzipping the dataset
!unzip '/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0'

Archive:  /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [12]:
#importing glove.6B
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-04-14 12:13:32--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-04-14 12:13:32--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-04-14 12:13:32--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [13]:
#unzipping the glove files
!unzip '/content/glove.6B.zip'

Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
#reading the dataset
dataset = r'/content/train.csv'

In [0]:
data = pd.read_csv(dataset)

In [16]:
data

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [0]:
#dropping qid column
data.drop(['qid'],axis=1,inplace=True)

In [18]:
data

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
...,...,...
1306117,What other technical skills do you need as a c...,0
1306118,Does MS in ECE have good job prospects in USA ...,0
1306119,Is foam insulation toxic?,0
1306120,How can one start a research project based on ...,0


In [0]:
#format our text samples and labels into tensors that can be fed into a neural network
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['question_text'])
sequences = tokenizer.texts_to_sequences(data['question_text'])

In [20]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 222161 unique tokens.


In [21]:
#finding the max_elnth of the sequence
max_len = 0
i= 0 
for i in range(len(sequences)):
    if len(sequences[i]) > max_len:
        max_len = len(sequences[i])
        #print(max_len)
        #print(sequences[i])
print(max_len)
        
    

190


In [22]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_data = pad_sequences(sequences, maxlen=max_len)


Found 222161 unique tokens.


In [23]:

labels = to_categorical(np.asarray(data['target']))
#labels = pd.DataFrame(labels)
#labels[0:24]

print(labels.shape)
print(X_data.shape)

(1306122, 2)
(1306122, 190)


In [24]:
# split the data into a training set and a validation set
indices = np.arange(X_data.shape[0])
np.random.shuffle(indices)
X_data = X_data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_data.shape[0])
print(nb_validation_samples)

x_train = X_data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

261224


In [25]:
x_train

array([[    0,     0,     0, ...,  9678,   213,  5781],
       [    0,     0,     0, ...,     7,   176, 29055],
       [    0,     0,     0, ...,    11,    45,    17],
       ...,
       [    0,     0,     0, ...,   288,  1604,  2046],
       [    0,     0,     0, ...,     1,   137,  3786],
       [    0,     0,     0, ...,  2198, 16254,   289]], dtype=int32)

In [26]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [28]:
#preparing the embedding layer
embeddings_index = {}
#f = os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')
f = glove_file
f = open(f,encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
#At this point we can leverage our embedding_index dictionary and our word_index to compute our embedding matrix:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [30]:
embedding_matrix.shape

(222162, 100)

In [0]:
#We load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [32]:
#Finally we can then build a small 1D convnet to solve our classification problem:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(2, activation='sigmoid')(x)

model = Model(sequence_input, outputs)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
#fitting
model.fit(x_train, y_train, validation_data=(x_val,y_val),
          epochs=5, batch_size=128)

Train on 1044898 samples, validate on 261224 samples
Epoch 1/5
1044898/1044898 [==============================] - 88s 85us/step - loss: 0.2019 - acc: 0.9427 - val_loss: 0.1973 - val_acc: 0.9439
Epoch 2/5
1044898/1044898 [==============================] - 82s 78us/step - loss: 0.1976 - acc: 0.9438 - val_loss: 0.1967 - val_acc: 0.9441
Epoch 3/5
1044898/1044898 [==============================] - 82s 79us/step - loss: 0.1954 - acc: 0.9446 - val_loss: 0.1995 - val_acc: 0.9440
Epoch 4/5
1044898/1044898 [==============================] - 82s 78us/step - loss: 0.1935 - acc: 0.9453 - val_loss: 0.1976 - val_acc: 0.9437
Epoch 5/5
1044898/1044898 [==============================] - 82s 78us/step - loss: 0.1915 - acc: 0.9462 - val_loss: 0.1983 - val_acc: 0.9436


In [0]:
#prediction on x_val
y_pred = model.predict(x_val)


In [0]:
y_pred [y_pred > 0.5] = 1.0
y_pred[y_pred <= 0.5] = 0.0

In [42]:
y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [0]:
#from  sklearn.metrics import roc_auc_score
#auc = roc_auc_score(y_val,y_pred)

In [0]:
#y_val

In [0]:
#auc

In [0]:
pred = pd.DataFrame(y_pred)


In [0]:
#exporting predictions to predictions file
pred.to_csv('predctions.csv',index=False)

In [0]:
#saving weights
#model.save_weights('prj_3_1D_CNN.h5')